### **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import windows


In [0]:
df = spark.read.format("parquet")\
      .load("abfss://bronze@databricksetesg.dfs.core.windows.net/orders")

In [0]:
display(df)

In [0]:
df.printSchema()

In [0]:
df = df.withColumnRenamed("_rescued_data","rescued_data")

In [0]:
df = df.drop("rescued_data")

In [0]:
df.display()

In [0]:


df = df.withColumn("order_date", to_timestamp(col("order_date")))
df.display()

In [0]:
df = df.withColumn("year",year(col("order_date")))
df.display()

df = df.withColumn("month", month(col("order_date")))
df.display()

In [0]:
df1 = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

In [0]:
df1 = df1.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

In [0]:
df1 = df1.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

### **Classes - OOP**

In [0]:
class windows:

    def dense_rank(self,df):

        df_dense_rank = df.withColumn("dense_rank",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

        return df_dense_rank

    def rank(self,df):

        df_rank = df.withColumn("rank",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

        return df_rank

    def row_number(self,df):

        df_row_number = df.withColumn("row_number",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

        return df_row_number

In [0]:
df_new = df

In [0]:
df_new.display()

In [0]:
obj = windows()

In [0]:
obj.dense_rank(df_new).display()
obj.rank(df_new).display()
obj.row_number(df_new).display()

### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databricksetesg.dfs.core.windows.net/orders") 

### **Create Table in Gold layer(Silver catalog)**

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cat.silver.orders_silver
USING DELTA
LOCATION 'abfss://silver@databricksetesg.dfs.core.windows.net/orders'